In [1]:
import numpy as np
import os
from os import listdir
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [2]:
#Load the 2nd dataset
import pandas as pd

#The data set : https://www.kaggle.com/datasets/debanga/facial-expression-recognition-challenge?resource=download
#Rename the folder as faceEmotion_dataset_v2

# read the CSV file
df = pd.read_csv("faceEmotion_dataset_v2/icml_face_data.csv/icml_face_data.csv")

num_classes = 7

# print the first few rows of the DataFrame
print(df.head())

   emotion     Usage                                             pixels
0        0  Training  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1        0  Training  151 150 147 155 148 133 111 140 170 174 182 15...
2        2  Training  231 212 156 164 174 138 161 173 182 200 106 38...
3        4  Training  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4        6  Training  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [3]:
#Split between training and test
maskTraining = df[" Usage"] == "Training"
maskTest = [not elem for elem in maskTraining]

df_train = df.loc[maskTraining]
df_test = df.loc[maskTest]

In [8]:
def list_string_to_arrayImages(L, sizeX=48, sizeY=48):

    Images_Dataset = []
    for k in range(L.index[0], L.index[-1] + 1):
        # split the string into a list of strings
        # lst = df[" pixels"][k].split()
        lst = L[k].split()

        # convert the strings to integers using map() and a lambda function
        lst = list(map(lambda x: int(x), lst))

        Images_Dataset.append(lst)
    Images_Dataset = np.array(Images_Dataset)
    Images_Dataset = Images_Dataset.reshape(Images_Dataset.shape[0], sizeX , sizeY)
    Images_Dataset = Images_Dataset.astype('float32')
    Images_Dataset /= 255
    return Images_Dataset

In [9]:
#Proprocess dataset and get X_train and X_test
X_train = list_string_to_arrayImages(df_train[" pixels"])
X_test = list_string_to_arrayImages(df_test[" pixels"])

print((X_train).shape)
print((X_test).shape)


(28709, 48, 48)
(7178, 48, 48)


In [11]:
#get y_train and y_test
Labels_train = list(df_train["emotion"])
Labels_test = list(df_test["emotion"])

In [12]:
#change the shape of the object labels
y_train = to_categorical(Labels_train, num_classes) 
y_test = to_categorical(Labels_test, num_classes) 

# define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

predictions = model.predict(X_test)

# calculate the accuracy of the predictions
accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))

print('Accuracy:', accuracy)

Epoch 1/10
898/898 [==============================] - 85s 93ms/step - loss: 1.6230 - accuracy: 0.3700
Epoch 2/10
898/898 [==============================] - 73s 81ms/step - loss: 1.4489 - accuracy: 0.4520
Epoch 3/10
898/898 [==============================] - 60s 67ms/step - loss: 1.3662 - accuracy: 0.4911
Epoch 4/10
898/898 [==============================] - 47s 52ms/step - loss: 1.3074 - accuracy: 0.5112
Epoch 5/10
898/898 [==============================] - 31s 35ms/step - loss: 1.2569 - accuracy: 0.5329
Epoch 6/10
898/898 [==============================] - 33s 36ms/step - loss: 1.2117 - accuracy: 0.5516
Epoch 7/10
898/898 [==============================] - 38s 42ms/step - loss: 1.1702 - accuracy: 0.5680
Epoch 8/10
898/898 [==============================] - 31s 34ms/step - loss: 1.1300 - accuracy: 0.5822
Epoch 9/10
898/898 [==============================] - 30s 33ms/step - loss: 1.0948 - accuracy: 0.5975
Epoch 10/10
225/225 [==============================] - 3s 13ms/step
Accuracy: 0.47

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score

# calculate the precision, recall, and F1 score
precision = precision_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
recall = recall_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))

print('Precision:', precision)
print('Recall:', recall)
print('F1 score:', f1)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].